In [ ]:
from flashml import plot_dist
import random
x1 = [random.randint(0, 5) for _ in range(10)]
x2 = [random.randint(0, 200) for _x in range(1000)]
x3 = [None, *x2, None]
x4= [random.random() for _ in range(1000)]
x5 = [None, *x4, None]
x6 = ["hi there"  , "jonanatha", "hi there", "hi there "]
x7 = [None, *x6]
x8 = {"daniel": 10, "james": 10}
plot_dist(x1)
plot_dist(x2)
plot_dist(x3)
plot_dist(x4)
plot_dist(x5)
plot_dist(x6)
plot_dist(x7)       
plot_dist(x8)  

In [ ]:
from transformers import AutoTokenizer
from flashml.inspect import inspect_tokenizer_window

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
inspect_tokenizer_window(tokenizer)

In [ ]:
from flashml import host_mlflow


host_mlflow()

In [3]:
from flashml import BatchIterator

data = range(3, 1000)

it = BatchIterator(data, batch_size=10, gradient_accumulation_steps=2, eval_ratio=0.2)
for i in it:
    print(i)

Batch(index=0, is_optim_step_time=False, eval_time=False, save_time=False, step=(1, 99), value=[441, 980, 451, 716, 327, 418, 34, 576, 510, 401])
Batch(index=1, is_optim_step_time=True, eval_time=False, save_time=False, step=(2, 99), value=[111, 393, 435, 990, 554, 523, 877, 744, 749, 722])
Batch(index=2, is_optim_step_time=False, eval_time=False, save_time=False, step=(3, 99), value=[801, 355, 189, 674, 382, 519, 254, 675, 697, 606])
Batch(index=3, is_optim_step_time=True, eval_time=False, save_time=False, step=(4, 99), value=[379, 528, 306, 767, 400, 883, 619, 496, 40, 880])
Batch(index=4, is_optim_step_time=False, eval_time=False, save_time=False, step=(5, 99), value=[988, 530, 473, 86, 367, 893, 571, 504, 51, 413])
Batch(index=5, is_optim_step_time=True, eval_time=False, save_time=False, step=(6, 99), value=[299, 772, 183, 150, 846, 75, 840, 17, 870, 157])
Batch(index=6, is_optim_step_time=False, eval_time=False, save_time=False, step=(7, 99), value=[822, 462, 115, 395, 286, 489, 3